In [57]:
import torch
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import sys
sys.path.append('skip-thoughts.torch/pytorch')
from skipthoughts import UniSkip,BiSkip


In [58]:
import pandas as pd
from Vocabulary import Vocabulary, preprocess

In [59]:
def make_vocab(tokens):
    voc = Vocabulary(['<PAD>','<UNK>'])
    voc.add_tokens(tokens)
    print('vocab len is {}'.format(len(voc.w2idx)))
    return voc

In [60]:
def load_data(file='story_cloze_data/cloze_test_val__spring2016 - cloze_test_ALL_val.csv'):
    df= pd.read_csv(file)
    df = df.drop('InputStoryid',axis=1)
    targets = df['AnswerRightEnding']
    df = df.drop('AnswerRightEnding',axis=1)
    df = df.drop('InputSentence1',axis=1)
    df = df.drop('InputSentence2',axis=1)
    df = df.drop('InputSentence3',axis=1)
    
    voc_str= ''
    for index, row in df.iterrows():
        voc_str+=' '.join(list(row)) + ' '
        
    df['AnswerRightEnding'] = targets
    return df,make_vocab(preprocess(voc_str))

In [61]:
import sys
class NoContextDataset(Dataset):
    def __init__(self,file='story_cloze_data/cloze_test_val__spring2016 - cloze_test_ALL_val.csv',vocab=None,df=None):

        super().__init__()
        
        created_df, created_vocab = load_data(file)
        if df is None:
            df = created_df
        if vocab:
            self.vocab = vocab
        else:
            self.vocab = created_vocab
        #self.df = df
      
        
        self.dir_st = 'data/skip-thoughts'
        self.biskip = BiSkip(self.dir_st, self.vocab.convert_to_list()[1:])
        
        self.uniskip = UniSkip(self.dir_st, self.vocab.convert_to_list()[1:])
        
        
        self.data = self.make_data(df)
        
        
    def __getitem__(self, idx):
        """
        Args:
            idx
        Returns: skip thought embedding of ending and 0/1 if it is the right ending 

        """
        return self.data[idx]

    def __len__(self):
        """
        Returns len of the dataset
        """
        return len(self.data)
       
    def make_data(self, df):
        data = []
        total = df.index
        print('skip thought encoding dataset')
        for i in total:
            progress(i,len(total))
            endings =  self.gen_embbeding(df.at[i,'RandomFifthSentenceQuiz1'], df.at[i,'RandomFifthSentenceQuiz2'])
            if df.at[i,'AnswerRightEnding'] == 1:
                data.append((endings[0].detach().numpy(),1))
                data.append((endings[1].detach().numpy(),0))
            else:
                data.append((endings[0].detach().numpy(),0))
                data.append((endings[1].detach().numpy(),1))
        return data
    

    def zero_pad(self,l,n):
        l = (l + n * [0])[:n]
        return l
    
    def pad_input(self,a,b):
        ed = sorted([a,b],key=len)
        longer = ed[1]
        shorter = ed[0]
        padded = self.zero_pad( shorter,len(longer))
        if shorter == a:
            return padded,b
        else: return a,padded
        
    def gen_embbeding(self,sent1,sent2):
        sent1 = preprocess(sent1)
        sent2 = preprocess(sent2)
        #remove random n token that is in one sentence
        if 'n' in sent2:
            sent2.remove('n')
        encoded_end1 = self.vocab.get_sentence(sent1)
        encoded_end2 = self.vocab.get_sentence(sent2)
        a,b = self.pad_input(encoded_end1,encoded_end2)
        
        batch = torch.LongTensor([a,b]) 
        top_half = self.uniskip(batch)
        bottom_half = self.biskip(batch)
        combine_skip = torch.cat([top_half,bottom_half],dim=1) 
        return combine_skip    
    
def progress(count, total, status=''):
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)

    sys.stdout.write('[%s] %s%s ...%s\r' % (bar, percents, '%', status))
    sys.stdout.flush()

In [62]:
class NoContextModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        self.input = torch.nn.Linear(4800,256)
        self.hidden= torch.nn.Linear(256,64)
        self.output = torch.nn.Linear(64,2)
        
    def forward(self, inputs):
        hidden = torch.nn.functional.relu(self.input(inputs))
        hidden1 = torch.nn.functional.relu(self.hidden(hidden))
        output = self.output(hidden1)
        return output

In [63]:
def score(pair,model):
    '''true if model predicts right'''
    ending1, ending2 = pair
    if ending1[1] == 1:
        target = 1
    else:
        target =  2 
        
    ending1 = torch.tensor(ending1[0])
    ending2 = torch.tensor(ending2[0])
    if torch.cuda.is_available():
        model = model.cuda()
        ending1 = ending1.cuda()
        ending2 = ending2.cuda()
    res1 = model(ending1)
    res2 = model(ending2)
    softm = torch.nn.Softmax(dim=0)
    prob_end1_right = softm(res1)[1].item() 
    prob_end2_right = softm(res2)[1].item()
    
    if prob_end1_right > prob_end2_right:
        pred = 1
    else:
        pred = 2
    
    if pred == target:
        return True
    else: 
        #print(prob_end1_right,prob_end2_right,pred,target)
        return False
   

In [64]:
def compute_accuracy(data_set):
    num_right = 0
    for i in range(0,len(data_set),2):
        if score((data_set[i],data_set[i+1]),model):
            num_right+=1
    return num_right / (len(data_set)/2)
    

In [65]:
def save(name):
    torch.save(model.state_dict(), 'saved_models/{}.save'.format(name))

In [80]:
import time
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
plt.style.use('seaborn-paper')
batch_size = 128
lr = 0.01
num_epochs = 30
report_every = 1


In [67]:
from sklearn.model_selection import train_test_split
df, voc = load_data()
train, val = train_test_split(df, test_size=0.1,shuffle=False)


vocab len is 5303


In [68]:
train_data_set = NoContextDataset(df=train,vocab=voc)
data_loader = torch.utils.data.DataLoader(train_data_set, batch_size=batch_size, shuffle=True,num_workers = 0)

vocab len is 5303


/home/grant/Desktop/school/nlp/env/lib/python3.6/site-packages/torch/nn/modules/rnn.py:46: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.25 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


skip thought encoding dataset


In [69]:
val_data_set = NoContextDataset(file='story_cloze_data/cloze_test_val__spring2016 - cloze_test_ALL_val.csv',df=val)

vocab len is 5303
skip thought encoding dataset


In [93]:
model = NoContextModel()
if torch.cuda.is_available():
    model = model.cuda()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [86]:
tick = time.time()
name='NC_4_26_sgd_ep'
epoch_losses = []
epoch_accs = []
best_score= 0.5
for epoch_num in range(1, num_epochs + 1):
    batch_losses = []
    for i, batch in enumerate(data_loader):
        ### YOUR CODE BELOW ###
        # Zero the gradients
        optimizer.zero_grad()
        # Extract the inputs and the targets
        inputs, targets = batch
        # Transfer the inputs and the targets to GPUs, if available
        if torch.cuda.is_available():
            inputs = torch.FloatTensor(inputs.float()).cuda()
            targets = torch.LongTensor(targets).cuda()
        # Run the model
        outputs = model(inputs)

        # Compute the loss
        loss = criterion(outputs,targets)
        
        
        # Backpropagate the error
        loss.backward(retain_graph=True)
        # Update the parameters
        optimizer.step()
        
        # Append the loss
        batch_losses.append(float (loss))
        ### YOUR CODE ABOVE ###
        epoch_loss = np.mean(np.array(batch_losses))
        epoch_losses.append(epoch_loss)
        
        acc = compute_accuracy(val_data_set)
        epoch_accs.append(acc)
        

        if epoch_num % report_every == 0:
            tock = time.time()
            print("Epoch {}. Loss {:.4f}. accuracy {:.4f}. Elapsed {:.0f} seconds".format(epoch_num, epoch_loss,acc, tock-tick))
    
    epoch_acc = np.mean(np.array(epoch_accs))
    print('ep acc',epoch_acc)
    if epoch_acc > best_score:
            best_score = epoch_acc
            print('new best saving model')
            save('{}{}_best_acc_{:.2f}'.format(name,epoch_num,epoch_acc))
     
    #print('saving model')
    #save(name+ str(epoch_num))

print("Total time elapsed: {:.0f} minutes".format((tock-tick)/60))

Epoch 1. Loss 0.4957. accuracy 0.7660. Elapsed 1 seconds
Epoch 1. Loss 0.4908. accuracy 0.7660. Elapsed 1 seconds
Epoch 1. Loss 0.5005. accuracy 0.7606. Elapsed 1 seconds
Epoch 1. Loss 0.5009. accuracy 0.7606. Elapsed 2 seconds
Epoch 1. Loss 0.5155. accuracy 0.7660. Elapsed 2 seconds
Epoch 1. Loss 0.5223. accuracy 0.7660. Elapsed 3 seconds
Epoch 1. Loss 0.5196. accuracy 0.7660. Elapsed 3 seconds
Epoch 1. Loss 0.5153. accuracy 0.7606. Elapsed 4 seconds
Epoch 1. Loss 0.5104. accuracy 0.7606. Elapsed 4 seconds
Epoch 1. Loss 0.5068. accuracy 0.7660. Elapsed 5 seconds
Epoch 1. Loss 0.5083. accuracy 0.7660. Elapsed 5 seconds
Epoch 1. Loss 0.5105. accuracy 0.7660. Elapsed 5 seconds
Epoch 1. Loss 0.5117. accuracy 0.7660. Elapsed 6 seconds
Epoch 1. Loss 0.5131. accuracy 0.7606. Elapsed 6 seconds
Epoch 1. Loss 0.5131. accuracy 0.7606. Elapsed 6 seconds
Epoch 1. Loss 0.5115. accuracy 0.7553. Elapsed 7 seconds
Epoch 1. Loss 0.5131. accuracy 0.7606. Elapsed 7 seconds
Epoch 1. Loss 0.5124. accuracy 

Epoch 6. Loss 0.4676. accuracy 0.7500. Elapsed 45 seconds
Epoch 6. Loss 0.4771. accuracy 0.7500. Elapsed 46 seconds
Epoch 6. Loss 0.4823. accuracy 0.7500. Elapsed 46 seconds
Epoch 6. Loss 0.4872. accuracy 0.7500. Elapsed 46 seconds
Epoch 6. Loss 0.4821. accuracy 0.7500. Elapsed 47 seconds
Epoch 6. Loss 0.4833. accuracy 0.7500. Elapsed 47 seconds
Epoch 6. Loss 0.4874. accuracy 0.7447. Elapsed 47 seconds
Epoch 6. Loss 0.4891. accuracy 0.7447. Elapsed 48 seconds
Epoch 6. Loss 0.4861. accuracy 0.7447. Elapsed 48 seconds
Epoch 6. Loss 0.4874. accuracy 0.7447. Elapsed 48 seconds
Epoch 6. Loss 0.4885. accuracy 0.7447. Elapsed 49 seconds
Epoch 6. Loss 0.4921. accuracy 0.7447. Elapsed 49 seconds
Epoch 6. Loss 0.4913. accuracy 0.7447. Elapsed 49 seconds
Epoch 6. Loss 0.4916. accuracy 0.7447. Elapsed 49 seconds
Epoch 6. Loss 0.4890. accuracy 0.7447. Elapsed 50 seconds
Epoch 6. Loss 0.4915. accuracy 0.7447. Elapsed 50 seconds
Epoch 6. Loss 0.4911. accuracy 0.7447. Elapsed 50 seconds
Epoch 6. Loss 

Epoch 11. Loss 0.4558. accuracy 0.7394. Elapsed 85 seconds
Epoch 11. Loss 0.4592. accuracy 0.7394. Elapsed 85 seconds
Epoch 11. Loss 0.4612. accuracy 0.7394. Elapsed 86 seconds
Epoch 11. Loss 0.4614. accuracy 0.7447. Elapsed 86 seconds
Epoch 11. Loss 0.4612. accuracy 0.7447. Elapsed 86 seconds
Epoch 11. Loss 0.4601. accuracy 0.7394. Elapsed 87 seconds
Epoch 11. Loss 0.4595. accuracy 0.7447. Elapsed 87 seconds
Epoch 11. Loss 0.4640. accuracy 0.7447. Elapsed 87 seconds
Epoch 11. Loss 0.4638. accuracy 0.7394. Elapsed 88 seconds
Epoch 11. Loss 0.4639. accuracy 0.7394. Elapsed 88 seconds
Epoch 11. Loss 0.4657. accuracy 0.7394. Elapsed 88 seconds
Epoch 11. Loss 0.4670. accuracy 0.7394. Elapsed 89 seconds
Epoch 11. Loss 0.4685. accuracy 0.7394. Elapsed 89 seconds
Epoch 11. Loss 0.4682. accuracy 0.7394. Elapsed 89 seconds
Epoch 11. Loss 0.4669. accuracy 0.7394. Elapsed 90 seconds
Epoch 11. Loss 0.4689. accuracy 0.7394. Elapsed 90 seconds
Epoch 11. Loss 0.4698. accuracy 0.7394. Elapsed 90 secon

Epoch 16. Loss 0.4559. accuracy 0.7340. Elapsed 127 seconds
Epoch 16. Loss 0.4584. accuracy 0.7340. Elapsed 127 seconds
Epoch 16. Loss 0.4564. accuracy 0.7394. Elapsed 127 seconds
Epoch 16. Loss 0.4558. accuracy 0.7340. Elapsed 128 seconds
Epoch 16. Loss 0.4545. accuracy 0.7340. Elapsed 128 seconds
Epoch 16. Loss 0.4522. accuracy 0.7340. Elapsed 129 seconds
Epoch 16. Loss 0.4490. accuracy 0.7287. Elapsed 130 seconds
Epoch 16. Loss 0.4487. accuracy 0.7287. Elapsed 130 seconds
Epoch 16. Loss 0.4484. accuracy 0.7287. Elapsed 130 seconds
Epoch 16. Loss 0.4480. accuracy 0.7340. Elapsed 131 seconds
Epoch 16. Loss 0.4481. accuracy 0.7287. Elapsed 131 seconds
Epoch 16. Loss 0.4490. accuracy 0.7287. Elapsed 132 seconds
Epoch 16. Loss 0.4493. accuracy 0.7287. Elapsed 132 seconds
Epoch 16. Loss 0.4473. accuracy 0.7287. Elapsed 132 seconds
Epoch 16. Loss 0.4487. accuracy 0.7287. Elapsed 133 seconds
Epoch 16. Loss 0.4461. accuracy 0.7287. Elapsed 133 seconds
Epoch 16. Loss 0.4447. accuracy 0.7394. 

Epoch 21. Loss 0.4199. accuracy 0.7287. Elapsed 170 seconds
Epoch 21. Loss 0.4231. accuracy 0.7287. Elapsed 170 seconds
Epoch 21. Loss 0.4209. accuracy 0.7287. Elapsed 171 seconds
Epoch 21. Loss 0.4197. accuracy 0.7287. Elapsed 171 seconds
Epoch 21. Loss 0.4182. accuracy 0.7287. Elapsed 171 seconds
Epoch 21. Loss 0.4173. accuracy 0.7340. Elapsed 172 seconds
Epoch 21. Loss 0.4179. accuracy 0.7394. Elapsed 172 seconds
Epoch 21. Loss 0.4171. accuracy 0.7234. Elapsed 172 seconds
Epoch 21. Loss 0.4160. accuracy 0.7287. Elapsed 173 seconds
Epoch 21. Loss 0.4191. accuracy 0.7234. Elapsed 173 seconds
Epoch 21. Loss 0.4209. accuracy 0.7287. Elapsed 173 seconds
Epoch 21. Loss 0.4200. accuracy 0.7340. Elapsed 173 seconds
Epoch 21. Loss 0.4211. accuracy 0.7287. Elapsed 174 seconds
Epoch 21. Loss 0.4219. accuracy 0.7234. Elapsed 174 seconds
Epoch 21. Loss 0.4236. accuracy 0.7234. Elapsed 174 seconds
Epoch 21. Loss 0.4224. accuracy 0.7234. Elapsed 175 seconds
Epoch 21. Loss 0.4243. accuracy 0.7074. 

Epoch 26. Loss 0.3942. accuracy 0.7128. Elapsed 211 seconds
Epoch 26. Loss 0.3925. accuracy 0.7128. Elapsed 211 seconds
Epoch 26. Loss 0.3931. accuracy 0.7181. Elapsed 211 seconds
Epoch 26. Loss 0.3935. accuracy 0.7074. Elapsed 212 seconds
Epoch 26. Loss 0.3967. accuracy 0.7181. Elapsed 212 seconds
Epoch 26. Loss 0.3975. accuracy 0.7128. Elapsed 213 seconds
Epoch 26. Loss 0.3976. accuracy 0.7181. Elapsed 213 seconds
Epoch 26. Loss 0.3974. accuracy 0.7128. Elapsed 213 seconds
Epoch 26. Loss 0.3952. accuracy 0.7074. Elapsed 214 seconds
Epoch 26. Loss 0.3965. accuracy 0.7128. Elapsed 214 seconds
Epoch 26. Loss 0.3973. accuracy 0.7128. Elapsed 215 seconds
Epoch 26. Loss 0.3993. accuracy 0.7128. Elapsed 215 seconds
Epoch 26. Loss 0.3992. accuracy 0.7181. Elapsed 215 seconds
Epoch 26. Loss 0.4033. accuracy 0.7181. Elapsed 216 seconds
Epoch 26. Loss 0.4033. accuracy 0.7181. Elapsed 216 seconds
Epoch 26. Loss 0.4028. accuracy 0.7181. Elapsed 216 seconds
Epoch 26. Loss 0.4007. accuracy 0.7074. 

In [ ]:

#train_data_set = None
test_data_set = NoContextDataset(file='story_cloze_data/cloze_test_test__spring2016 - cloze_test_ALL_test.csv',vocab=voc)

In [21]:
#test_data_set= None

In [90]:
def load(name):
    l_model = NoContextModel()
    l_model.load_state_dict(torch.load('saved_models/{}.save'.format(name)))
    return l_model
model = load('NC_4_26_sgd_ep30_best_acc_0.70')
#print('model',model)

In [92]:
compute_accuracy(test_data_set)

0.6932121859967931